# TL;DR

This solution is an ensemble of a few models:

- Model 1: roberta-base, further pre-trained by [Maunish dave](https://www.kaggle.com/maunish), and available in [this dataset](https://www.kaggle.com/maunish/clrp-roberta-base). Inference code was shared in [this notebook](https://www.kaggle.com/andretugan/pre-trained-roberta-solution-in-pytorch), by [Andrey Tuganov](https://www.kaggle.com/andretugan).

- Model 2: Almost the same code as Model 1, roberta-base. I changed the architecture: changed the number of neurons, added one Linear layer, and changed the activation function. This is the best CV I got. Training was done in another notebook.

- Model 3: Inspired by [this work](https://www.kaggle.com/maunish/clrp-roberta-svm), also from [Maunish dave](https://www.kaggle.com/maunish), I extracted the embeddings from Model 1 and used ExtraTreesRegressor to predict the target. 10 seeds were used here.

- Model 4: Code is from [this notebook](https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-infer-3), published by [torch](https://www.kaggle.com/rhtsingh). He used an ensemble of three models, but I only used his second checkpoint, which is a roberta-large, also further pre-trained.

- Model 5: roberta-large with MeanPooling shared in [this notebook](https://www.kaggle.com/rajat95gupta/clrp-ensemble-3x-inference) by [Rajat Gupta](https://www.kaggle.com/rajat95gupta). Two seeds were used in this inference.

If you like this solution, please don't forget to upvote the all the works mentioned above!

# Model 1

Roberta-base, further pre-trained by [Maunish dave](https://www.kaggle.com/maunish), and available in [this dataset](https://www.kaggle.com/maunish/clrp-roberta-base). Inference code was shared in [this notebook](https://www.kaggle.com/andretugan/pre-trained-roberta-solution-in-pytorch), by [Andrey Tuganov](https://www.kaggle.com/andretugan).

In [ ]:
import os
import math
import random
import time
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig

from sklearn.model_selection import KFold
from sklearn.svm import SVR

import gc
gc.enable()

In [ ]:
BATCH_SIZE = 32
MAX_LEN = 248
EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
ROBERTA_PATH = "/kaggle/input/roberta-base"
TOKENIZER_PATH = "/kaggle/input/roberta-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
test_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
submission_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

## Dataset

In [ ]:
class LitDataset(Dataset):
    def __init__(self, df, inference_only=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only
        self.text = df.excerpt.tolist()
        #self.text = [text.replace("\n", " ") for text in self.text]
        
        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32)        
    
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return (input_ids, attention_mask)            
        else:
            target = self.target[index]
            return (input_ids, attention_mask, target)

## Model
The model is inspired by the one from [Maunish](https://www.kaggle.com/maunish/clrp-roberta-svm).

In [ ]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config)  
            
        self.attention = nn.Sequential(            
            nn.Linear(768, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(768, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

In [ ]:
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    with torch.no_grad():
        for batch_num, (input_ids, attention_mask) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask)                        

            result[index : index + pred.shape[0]] = pred.flatten().to("cpu")
            index += pred.shape[0]

    return result

## Model 1 Inference

In [ ]:
test_dataset = LitDataset(test_df, inference_only=True)

In [ ]:
%%time

NUM_MODELS = 5

all_predictions = np.zeros((NUM_MODELS, len(test_df)))



test_dataset = LitDataset(test_df, inference_only=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=False, shuffle=False, num_workers=2)

for model_index in range(NUM_MODELS):            
    model_path = f"../input/commonlit-roberta-0467/model_{model_index + 1}.pth"
    print(f"\nUsing {model_path}")
                        
    model = LitModel()
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))    
    model.to(DEVICE)
        
    all_predictions[model_index] = predict(model, test_loader)
            
    del model
    gc.collect()

In [ ]:
model1_predictions = all_predictions.mean(axis=0)

# Model 2

Almost the same code as Model 1, roberta-base. I changed the architecture: changed the number of neurons, added one Linear layer, and changed the activation function. This is the best CV I got.

In [ ]:
class LitModel1_2(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config)  
            
        self.attention = nn.Sequential(            
            nn.Linear(768, 1024),  # (768, 512)
            nn.SELU(),    # Tanh
            nn.Linear(1024, 512),  #  added
            nn.SELU(),     # added   PReLU, ReLU, RELU6, RReLU, SELU, CELU, GELU, SiLU
            nn.Linear(512, 1), # (512, 1)
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(768, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

In [ ]:
%%time

NUM_MODELS = 5

all_predictions = np.zeros((NUM_MODELS, len(test_df)))



test_dataset = LitDataset(test_df, inference_only=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=False, shuffle=False, num_workers=2)

for model_index in range(NUM_MODELS):            
    model_path = f"../input/model17681024512/model_{model_index + 1}.pth"
    print(f"\nUsing {model_path}")
                        
    model = LitModel1_2()
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))    
    model.to(DEVICE)
        
    all_predictions[model_index] = predict(model, test_loader)
            
    del model
    gc.collect()

In [ ]:
model2_predictions = all_predictions.mean(axis=0)

# Model 3

Inspired by [this work](https://www.kaggle.com/maunish/clrp-roberta-svm), also from [Maunish dave](https://www.kaggle.com/maunish), I extracted the embeddings from Model 1 and used ExtraTreesRegressor to predict the target. 15 seeds were used here.

In [ ]:
class LitModelTabular(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config)  
            
        self.attention = nn.Sequential(            
            nn.Linear(768, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(768, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
#         return self.regressor(context_vector) # original
        return context_vector

In [ ]:
def get_embeddings(df, path = "../input/commonlit-roberta-0467/model_2.pth", plot_losses=True, verbose=True):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"{device} is used")
            
    model = LitModelTabular()
    model.load_state_dict(torch.load(path, map_location=DEVICE)) # path
    model.to(device)
    model.eval()
    
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH) # '../input/roberta-base'
    
    ds = LitDataset(df, inference_only=True)
    dl = DataLoader(ds,
                  batch_size = BATCH_SIZE,
                  shuffle=False,
                  num_workers = 4,
                  pin_memory=True,
                  drop_last=False
                 )
        
    embeddings = list()
    with torch.no_grad():
        for input_ids, attention_mask in tqdm(dl):
            inputs = input_ids.to(device)
            att = attention_mask.to(device)
#             tgt = target[i].to(device)
            # {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
            outputs = model(inputs, att)
            outputs = outputs.detach().cpu().numpy()
            embeddings.extend(outputs)
    return np.array(embeddings)

In [ ]:
train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
train_data = train_data.loc[:, ['excerpt', 'target']]
target = train_data['target'].to_numpy()

# train_data

In [ ]:
%%time

train_embeddings1 =  get_embeddings(train_data,'../input/commonlit-roberta-0467/model_1.pth')
test_embeddings1 = get_embeddings(test_df,'../input/commonlit-roberta-0467/model_1.pth')

train_embeddings2 =  get_embeddings(train_data,'../input/commonlit-roberta-0467/model_2.pth')
test_embeddings2 = get_embeddings(test_df,'../input/commonlit-roberta-0467/model_2.pth')

train_embeddings3 =  get_embeddings(train_data,'../input/commonlit-roberta-0467/model_3.pth')
test_embeddings3 = get_embeddings(test_df,'../input/commonlit-roberta-0467/model_3.pth')

train_embeddings4 =  get_embeddings(train_data,'../input/commonlit-roberta-0467/model_4.pth')
test_embeddings4 = get_embeddings(test_df,'../input/commonlit-roberta-0467/model_4.pth')

train_embeddings5 =  get_embeddings(train_data,'../input/commonlit-roberta-0467/model_5.pth')
test_embeddings5 = get_embeddings(test_df,'../input/commonlit-roberta-0467/model_5.pth')

In [ ]:
train_embeddings1.shape ,test_embeddings1.shape

## ExtraTreesRegressor

[ExtraTreesRegressor docs](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html)

In [ ]:
%%time

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, RobustScaler

def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

def get_preds_etr(X, y, X_test, seed, nfolds=5):
    scores = list()
    preds = np.zeros((X_test.shape[0]))
    
    # Don't scale targets, only features 
    scaler = MinMaxScaler() # RobustScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    scaler.fit(X_test)
    X_test_scaled = scaler.transform(X_test)
    
    X, X_test = X_scaled, X_test_scaled
    
    kfold = KFold(n_splits=nfolds, shuffle=True, random_state=seed)
    for k, (train_idx, valid_idx) in enumerate(kfold.split(X, y)):
        model = ExtraTreesRegressor(random_state=seed)
        X_train, y_train = X[train_idx], y[train_idx]
        X_valid, y_valid = X[valid_idx], y[valid_idx]
        
        model.fit(X_train,y_train)
        prediction = model.predict(X_valid)
        score = rmse_score(prediction,y_valid)
        print(f'Fold {k} , rmse score: {score}')
        scores.append(score)
        preds += model.predict(X_test)
        
    print("mean rmse",np.mean(scores))
    return np.array(preds)/nfolds


etr_preds1 = get_preds_etr(train_embeddings1, target, test_embeddings1, 123)
etr_preds2 = get_preds_etr(train_embeddings2, target, test_embeddings2, 123)
etr_preds3 = get_preds_etr(train_embeddings3, target, test_embeddings3, 123)
etr_preds4 = get_preds_etr(train_embeddings4, target, test_embeddings4, 123)
etr_preds5 = get_preds_etr(train_embeddings5, target, test_embeddings5, 123)

# other seed
etr_preds6 = get_preds_etr(train_embeddings1, target, test_embeddings1, 234)
etr_preds7 = get_preds_etr(train_embeddings2, target, test_embeddings2, 234)
etr_preds8 = get_preds_etr(train_embeddings3, target, test_embeddings3, 234)
etr_preds9 = get_preds_etr(train_embeddings4, target, test_embeddings4, 234)
etr_preds10 = get_preds_etr(train_embeddings5, target, test_embeddings5, 234)

#### 1 seed
# etr_preds = (etr_preds1 + etr_preds2 + etr_preds3 + etr_preds4 + etr_preds5)/5

#### 2 seeds
etr_preds = (etr_preds1 + etr_preds2 + etr_preds3 + etr_preds4 + etr_preds5 + etr_preds6 + etr_preds7 + etr_preds8 + etr_preds9 + etr_preds10)/10

etr_preds

## SVM

[SVR docs](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

In [ ]:
%%time

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.svm import SVR

def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))


def get_preds_svm(X, y, X_test, seed, nfolds=5):
    scores = list()
    preds = np.zeros((X_test.shape[0]))
    
    # Don't scale targets, only features 
    scaler = MinMaxScaler() # RobustScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    scaler.fit(X_test)
    X_test_scaled = scaler.transform(X_test)
    
    kfold = KFold(n_splits=nfolds, shuffle=True, random_state=seed)
    for k, (train_idx, valid_idx) in enumerate(kfold.split(X, y)):
        model = SVR(C=10, kernel='rbf', gamma='auto')
        X_train, y_train = X[train_idx], y[train_idx]
        X_valid, y_valid = X[valid_idx], y[valid_idx]
        
        model.fit(X_train,y_train)
        prediction = model.predict(X_valid)
        score = rmse_score(prediction,y_valid)
        print(f'Fold {k} , rmse score: {score}')
        scores.append(score)
        preds += model.predict(X_test)
        
    print("mean rmse",np.mean(scores))
    return np.array(preds)/nfolds


svm_preds1 = get_preds_svm(train_embeddings1, target, test_embeddings1, 678)
svm_preds2 = get_preds_svm(train_embeddings2, target, test_embeddings2, 678)
svm_preds3 = get_preds_svm(train_embeddings3, target, test_embeddings3, 678)
svm_preds4 = get_preds_svm(train_embeddings4, target, test_embeddings4, 678)
svm_preds5 = get_preds_svm(train_embeddings5, target, test_embeddings5, 678)

# other seed
svm_preds6 = get_preds_svm(train_embeddings1, target, test_embeddings1, 789)
svm_preds7 = get_preds_svm(train_embeddings2, target, test_embeddings2, 789)
svm_preds8 = get_preds_svm(train_embeddings3, target, test_embeddings3, 789)
svm_preds9 = get_preds_svm(train_embeddings4, target, test_embeddings4, 789)
svm_preds10 = get_preds_svm(train_embeddings5, target, test_embeddings5, 789)


#### 1 seed
# svm_preds = (svm_preds1 + svm_preds2 + svm_preds3 + svm_preds4 + svm_preds5)/5

#### 2 seeds
svm_preds = (svm_preds1 + svm_preds2 + svm_preds3 + svm_preds4 + svm_preds5 + svm_preds6 + svm_preds7 + svm_preds8 + svm_preds9 + svm_preds10)/10

svm_preds

# Model 4

Code is from [this notebook](https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-infer-3), published by [torch](https://www.kaggle.com/rhtsingh). He used an ensemble of three models, but I only used his second checkpoint, which is a roberta-large, also further pre-trained.

In [ ]:
test = test_df

from glob import glob
import os
import matplotlib.pyplot as plt
import json
from collections import defaultdict


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.optimizer import Optimizer
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import (
    Dataset, DataLoader, 
    SequentialSampler, RandomSampler
)
from transformers import RobertaConfig
from transformers import (
    get_cosine_schedule_with_warmup, 
    get_cosine_with_hard_restarts_schedule_with_warmup
)
from transformers import RobertaTokenizer
from transformers import RobertaModel
from IPython.display import clear_output
from tqdm import tqdm, trange

def convert_examples_to_features(data, tokenizer, max_len, is_test=False):
    data = data.replace('\n', '')
    tok = tokenizer.encode_plus(
        data, 
        max_length=max_len, 
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True
    )
    curr_sent = {}
    padding_length = max_len - len(tok['input_ids'])
    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + \
        ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + \
        ([0] * padding_length)
    return curr_sent

class DatasetRetriever(Dataset):
    def __init__(self, data, tokenizer, max_len, is_test=False):
        self.data = data
        self.excerpts = self.data.excerpt.values.tolist()
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        if not self.is_test:
            excerpt, label = self.excerpts[item], self.targets[item]
            features = convert_examples_to_features(
                excerpt, self.tokenizer, 
                self.max_len, self.is_test
            )
            return {
                'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
                'token_type_ids':torch.tensor(features['token_type_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
                'label':torch.tensor(label, dtype=torch.double),
            }
        else:
            excerpt = self.excerpts[item]
            features = convert_examples_to_features(
                excerpt, self.tokenizer, 
                self.max_len, self.is_test
            )
            return {
                'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
                'token_type_ids':torch.tensor(features['token_type_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
            }

class CommonLitModel(nn.Module):
    def __init__(
        self, 
        model_name, 
        config,  
        multisample_dropout=False,
        output_hidden_states=False
    ):
        super(CommonLitModel, self).__init__()
        self.config = config
        self.roberta = RobertaModel.from_pretrained(
            model_name, 
            output_hidden_states=output_hidden_states
        )
        self.layer_norm = nn.LayerNorm(config.hidden_size)
        if multisample_dropout:
            self.dropouts = nn.ModuleList([
                nn.Dropout(0.5) for _ in range(5)
            ])
        else:
            self.dropouts = nn.ModuleList([nn.Dropout(0.3)])
        self.regressor = nn.Linear(config.hidden_size, 1)
        self._init_weights(self.layer_norm)
        self._init_weights(self.regressor)
 
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
 
    def forward(
        self, 
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        sequence_output = outputs[1]
        sequence_output = self.layer_norm(sequence_output)
 
        # multi-sample dropout
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                logits = self.regressor(dropout(sequence_output))
            else:
                logits += self.regressor(dropout(sequence_output))
        
        logits /= len(self.dropouts)
 
        # calculate loss
        loss = None
        if labels is not None:
            loss_fn = torch.nn.MSELoss()
            logits = logits.view(-1).to(labels.dtype)
            loss = torch.sqrt(loss_fn(logits, labels.view(-1)))
        
        output = (logits,) + outputs[1:]
        return ((loss,) + output) if loss is not None else output

def make_model(model_name, num_labels=1):
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    config = RobertaConfig.from_pretrained(model_name)
    config.update({'num_labels':num_labels})
    model = CommonLitModel(model_name, config=config)
    return model, tokenizer

def make_loader(
    data, 
    tokenizer, 
    max_len,
    batch_size,
):
    
    test_dataset = DatasetRetriever(data, tokenizer, max_len, is_test=True)
    test_sampler = SequentialSampler(test_dataset)
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size // 2, 
        sampler=test_sampler, 
        pin_memory=False, 
        drop_last=False, 
        num_workers=0
    )

    return test_loader

class Evaluator:
    def __init__(self, model, scalar=None):
        self.model = model
        self.scalar = scalar

    def evaluate(self, data_loader, tokenizer):
        preds = []
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch_idx, batch_data in enumerate(data_loader):
                input_ids, attention_mask, token_type_ids = batch_data['input_ids'], \
                    batch_data['attention_mask'], batch_data['token_type_ids']
                input_ids, attention_mask, token_type_ids = input_ids.cuda(), \
                    attention_mask.cuda(), token_type_ids.cuda()
                
                if self.scalar is not None:
                    with torch.cuda.amp.autocast():
                        outputs = self.model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids
                        )
                else:
                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids
                    )
                
                logits = outputs[0].detach().cpu().numpy().squeeze().tolist()
                preds += logits
        return preds

def config(fold, model_name, load_model_path):
    torch.manual_seed(2021)
    torch.cuda.manual_seed(2021)
    torch.cuda.manual_seed_all(2021)
    
    max_len = 250
    batch_size = 8

    model, tokenizer = make_model(
        model_name=model_name, 
        num_labels=1
    )
    model.load_state_dict(
        torch.load(f'{load_model_path}/model{fold}.bin')
    )
    test_loader = make_loader(
        test, tokenizer, max_len=max_len,
        batch_size=batch_size
    )

    if torch.cuda.device_count() >= 1:
        print('Model pushed to {} GPU(s), type {}.'.format(
            torch.cuda.device_count(), 
            torch.cuda.get_device_name(0))
        )
        model = model.cuda() 
    else:
        raise ValueError('CPU training is not supported')

    # scaler = torch.cuda.amp.GradScaler()
    scaler = None
    return (
        model, tokenizer, 
        test_loader, scaler
    )

def run(fold=0, model_name=None, load_model_path=None):
    model, tokenizer, \
        test_loader, scaler = config(fold, model_name, load_model_path)
    
    import time

    evaluator = Evaluator(model, scaler)

    test_time_list = []

    torch.cuda.synchronize()
    tic1 = time.time()

    preds = evaluator.evaluate(test_loader, tokenizer)

    torch.cuda.synchronize()
    tic2 = time.time() 
    test_time_list.append(tic2 - tic1)
    
    del model, tokenizer, test_loader, scaler
    gc.collect()
    torch.cuda.empty_cache()
    
    return preds

pred_df1 = pd.DataFrame()
pred_df2 = pd.DataFrame()
pred_df3 = pd.DataFrame()
for fold in tqdm(range(5)):
    pred_df1[f'fold{fold}'] = run(fold, '../input/roberta-base/', '../input/commonlit-roberta-base-i/')
    pred_df2[f'fold{fold+5}'] = run(fold, '../input/robertalarge/', '../input/roberta-large-itptfit/')
    pred_df3[f'fold{fold+10}'] = run(fold, '../input/robertalarge/', '../input/commonlit-roberta-large-ii/')

In [ ]:
pred_df1 = np.array(pred_df1)
pred_df2 = np.array(pred_df2)
pred_df3 = np.array(pred_df3)
model4_predictions = ((pred_df1.mean(axis=1)*0.3) + (pred_df2.mean(axis=1)*0.5) + (pred_df3.mean(axis=1) * 0.2))

# Model 5

Roberta-large with MeanPooling shared in [this notebook](https://www.kaggle.com/rajat95gupta/clrp-ensemble-3x-inference) by [Rajat Gupta](https://www.kaggle.com/rajat95gupta).

In [ ]:
%%time

import os
import numpy as np
import pandas as pd
import random

from transformers import AutoConfig, AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, logging

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, SequentialSampler, RandomSampler, DataLoader

from tqdm.notebook import tqdm

import gc; gc.enable()
from IPython.display import clear_output

from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
logging.set_verbosity_error()

INPUT_DIR = '../input/commonlitreadabilityprize'
MODEL_DIR = '../input/roberta-transformers-pytorch/roberta-large'
CHECKPOINT_DIR = '../input/clrp-mean-pooling/'

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_LENGTH = 248
TEST_BATCH_SIZE = 1
HIDDEN_SIZE = 1024

NUM_FOLDS = 5
SEEDS = [113, 71]

test = pd.read_csv(os.path.join(INPUT_DIR, 'test.csv'))

class MeanPoolingModel(nn.Module):
    
    def __init__(self, model_name):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.linear = nn.Linear(HIDDEN_SIZE, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        outputs = self.model(input_ids, attention_mask)
        last_hidden_state = outputs[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        logits = self.linear(mean_embeddings)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds

def get_test_loader(data):

    x_test = data.excerpt.tolist()
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

    encoded_test = tokenizer.batch_encode_plus(
        x_test, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        truncation=True,
        max_length=MAX_LENGTH, 
        return_tensors='pt'
    )

    dataset_test = TensorDataset(
        encoded_test['input_ids'],
        encoded_test['attention_mask']
    )

    dataloader_test = DataLoader(
        dataset_test,
        sampler = SequentialSampler(dataset_test),
        batch_size=TEST_BATCH_SIZE
    )
    
    return dataloader_test

test_dataloader = get_test_loader(test)

all_predictions = []
for seed in SEEDS:
    
    fold_predictions = []
    
    for fold in tqdm(range(NUM_FOLDS)):
        model_path = f"model_{seed + 1}_{fold + 1}.pth"
        
        print(f"\nUsing {model_path}")
        
        model_path = CHECKPOINT_DIR + f"model_{seed + 1}_{fold + 1}.pth"
        model = MeanPoolingModel(MODEL_DIR)
        model.load_state_dict(torch.load(model_path)) 
        model.to(DEVICE)
        model.eval()

        predictions = []
        for batch in test_dataloader:

            batch = tuple(b.to(DEVICE) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         None,
                     }

     
            preds = model(**inputs).item()
            predictions.append(preds)
            
        del model 
        gc.collect()
            
        fold_predictions.append(predictions)
    all_predictions.append(np.mean(fold_predictions, axis=0).tolist())
    
model5_predictions = np.mean(all_predictions,axis=0)


# predictions = model1_predictions * 0.5 + model2_predictions * 0.3 + model3_predictions * 0.2  
# predictions

# results = pd.DataFrame(np.vstack((model1_predictions, model2_predictions, model3_predictions, predictions)).transpose(), 
#                        columns=['model1','model2','model3','ensemble'])

# results

In [ ]:
model5_predictions

# Ensemble

In [ ]:
predictions = model1_predictions * 0.15 + \
              model2_predictions * 0.25 + \
              etr_preds * 0.10 + \
              svm_preds * 0.00 + \
              pred_df1.mean(axis=1) * 0.00 + \
              pred_df2.mean(axis=1) * 0.10 + \
              pred_df3.mean(axis=1) * 0.00 + \
              model5_predictions * 0.40

In [ ]:
submission_df.target = predictions
print(submission_df)
submission_df.to_csv("submission.csv", index=False)